In [18]:
### create_sites.py
import pynetbox
import yaml
 
ENV_FILE = "yml_examples/env.yml"
SITES_FILE = "yml_examples/sites.yml"
 
with open(ENV_FILE) as file:
    env_vars = yaml.safe_load(file)
     
with open(SITES_FILE) as file:
    sites_to_load = yaml.safe_load(file)
     
nb_conn = pynetbox.api(url=env_vars['netbox_url'])
 
token = nb_conn.create_token(env_vars['username'], env_vars['password'])
 
for site in sites_to_load:
    name = site['name'].upper()
    slug = site['name'].lower()
    queried_site = nb_conn.dcim.sites.get(name=name)
    if queried_site:
        print(f"Site {site['name']} already exists.")
        continue
    print(f"Adding {site['name']} to Netbox.")
    constructed_site = {"name": name, "slug": slug}
    if "description" in site.keys():
        constructed_site['description'] = site['description']
    if "physical_address" in site.keys():
        constructed_site['physical_address'] = site['physical_address']
    result = nb_conn.dcim.sites.create(constructed_site)
     
token.delete()

Site CUB already exists.


True

In [19]:
### create_devices.py
import pynetbox
import yaml
 
ENV_FILE = "yml_examples/env.yml"
DEVICES_FILE = "yml_examples/device.yml"
 
with open(ENV_FILE) as file:
    env_vars = yaml.safe_load(file)
     
with open(DEVICES_FILE) as file:
    devices_to_load = yaml.safe_load(file)
     
nb_conn = pynetbox.api(url=env_vars['netbox_url'])
 
token = nb_conn.create_token(env_vars['username'], env_vars['password'])
 
valid_devices_status = []
for choice in nb_conn.dcim.devices.choices()['status']:
    valid_devices_status.append(choice['value'])

for device in devices_to_load:
    name = device['name'].upper()
    slug = device['name'].lower()
     
    queried_device = nb_conn.dcim.devices.get(name=name)
    if queried_device:
        print(f"The device {name} already exists. Skipping.")
        continue
     
    dev_type = device['type'].upper()
    queried_type = nb_conn.dcim.device_types.get(model=dev_type)
    if isinstance(queried_type, type(None)):
        print(f"The type {dev_type} does not exist. Skipping.")
        continue
     
    dev_role = device['role']
    queried_role = nb_conn.dcim.device_roles.get(name=dev_role)
    if isinstance(queried_role, type(None)):
        print(f"The role {dev_role} does not exist. Skipping.")
        continue
     
    site = device['site'].upper()
    queried_site = nb_conn.dcim.sites.get(name=site)
    if isinstance(queried_site, type(None)):
        print(f"The site {site} does not exist. Skipping.")
        continue
     
     
    constructed_device = {"name": name, "slug": slug, "site": queried_site.id, "device_role": queried_role.id, "device_type": queried_type.id}
    if "description" in device.keys():
        constructed_device['description'] = device['description']
    if "status" in device.keys():
        if device['status'] in valid_devices_status:
            constructed_device['status'] = device['status']
        else:
            print(f"The status of {device['status']} isn't valid. Skipping.")
            continue
    print(f"Adding {device['name']} to Netbox.")
    result = nb_conn.dcim.devices.create(constructed_device)
     
token.delete()

Adding CEOS1 to Netbox.
Adding CEOS2 to Netbox.
Adding CRPD1 to Netbox.
Adding CRPD2 to Netbox.


True

In [17]:
import pynetbox
import yaml
 
ENV_FILE = "yml_examples/env.yml"
 
with open(ENV_FILE) as file:
    env_vars = yaml.safe_load(file)
     
nb_conn = pynetbox.api(url=env_vars['netbox_url'])
 
token = nb_conn.create_token(env_vars['username'], env_vars['password'])

my_dev = nb_conn.dcim.devices.filter(site="CUB")

In [42]:
import requests

my_token = "848317b1cbe256959db547b309e35e46aeeeb6ac"
head = {'Authorization': 'token {}'.format(my_token)}
null = ""

ceos_config = {
    "header": [
        "! device: ceos1 (cEOSLab, EOS-4.28.3M-28837868.4283M (engineering build))\n!\n"
    ],
    "comments": [],
    "cmds": {
        "management api netconf": {
            "comments": [],
            "cmds": {
                "transport ssh default": {
                    "comments": [],
                    "cmds": {}
                }
            }
        },
        "management api gnmi": {
            "comments": [],
            "cmds": {
                "transport grpc default": {
                    "comments": [],
                    "cmds": {}
                }
            }
        },
        "router bgp 65000": {
            "comments": [],
            "cmds": {
                "neighbor 10.10.0.1 maximum-routes 12000": null,
                "address-family ipv4": {
                    "comments": [],
                    "cmds": {
                        "neighbor 10.10.0.1 activate": null,
                        "neighbor 10.10.1.1 activate": null
                    }
                },
                "neighbor 10.10.1.1 maximum-routes 12000": null,
                "neighbor 10.10.0.1 remote-as 65003": null,
                "neighbor 2001:db8:10:10:1::1 remote-as 65001": null,
                "neighbor 2001:db8:10:10::1 remote-as 65003": null,
                "redistribute connected": null,
                "neighbor 10.10.1.1 remote-as 65001": null,
                "address-family ipv6": {
                    "comments": [],
                    "cmds": {
                        "neighbor 2001:db8:10:10::1 activate": null,
                        "neighbor 2001:db8:10:10:1::1 activate": null
                    }
                }
            }
        },
        "interface Loopback0": {
            "comments": [],
            "cmds": {
                "ipv6 address 2001:db8:0:1::/128": null,
                "ip address 192.168.0.0/32": null
            }
        },
        "username admin privilege 15 role network-admin secret sha512 $6$Z98W49v6O15ZKGW2$DlAQUoSHjUe3gphPllQhF8TUkwTNGfewwXZTaSi9h5HrKi3k0CQx8GRSpyQ3twfpd5htv05f2ZPSjnPmysT//1": null,
        "transceiver qsfp default-mode 4x10G": null,
        "service routing protocols model multi-agent": null,
        "interface Ethernet1": {
            "comments": [],
            "cmds": {
                "ipv6 address 2001:db8:10:10::/127": null,
                "no switchport": null,
                "description ceos2:Ethernet1": null,
                "ip address 10.10.0.0/31": null
            }
        },
        "ipv6 unicast-routing": null,
        "management api http-commands": {
            "comments": [],
            "cmds": {
                "protocol http": null,
                "no shutdown": null
            }
        },
        "interface Ethernet2": {
            "comments": [],
            "cmds": {
                "no switchport": null,
                "ipv6 address 2001:db8:10:10:1::/127": null,
                "description srl1:ethernet-1/1": null,
                "ip address 10.10.1.0/31": null
            }
        },
        "ip routing": null,
        "interface Management0": {
            "comments": [],
            "cmds": {
                "ip address 172.22.0.2/24": null
            }
        },
        "spanning-tree mode mstp": null,
        "hostname ceos1": null,
        "no aaa root": null
    }
}
print(type(ceos_config))

<class 'dict'>


In [63]:
import json

f = open("yml_examples/current_configs")
data = json.load(f)
# print(data.keys())

dict_keys(['ceos1_config', 'ceos2_config'])


In [65]:
import pynetbox
import yaml
import json

f = open("yml_examples/current_configs")
data = json.load(f)
 
for key in data:
    dev_name = key
    config = data[key]
    r = requests.post('http://3.18.147.34:8000/api/extras/config-contexts/', json={
      "name": dev_name,
      "weight": 1000,
      "description": dev_name,
      "is_active": "true",
      "data":config
    },headers=head)
    print(f"Status Code: {r.status_code}, Response: {r.json()}")

Status Code: 201, Response: {'id': 7, 'url': 'http://3.18.147.34:8000/api/extras/config-contexts/7/', 'display': 'ceos1_config', 'name': 'ceos1_config', 'weight': 1000, 'description': 'ceos1_config', 'is_active': True, 'regions': [], 'site_groups': [], 'sites': [], 'locations': [], 'device_types': [], 'roles': [], 'platforms': [], 'cluster_types': [], 'cluster_groups': [], 'clusters': [], 'tenant_groups': [], 'tenants': [], 'tags': [], 'data': {'header': ['! device: ceos1 (cEOSLab, EOS-4.28.3M-28837868.4283M (engineering build))\n!\n'], 'comments': [], 'cmds': {'management api netconf': {'comments': [], 'cmds': {'transport ssh default': {'comments': [], 'cmds': {}}}}, 'management api gnmi': {'comments': [], 'cmds': {'transport grpc default': {'comments': [], 'cmds': {}}}}, 'router bgp 65000': {'comments': [], 'cmds': {'neighbor 10.10.0.1 maximum-routes 12000': None, 'address-family ipv4': {'comments': [], 'cmds': {'neighbor 10.10.0.1 activate': None, 'neighbor 10.10.1.1 activate': None

In [56]:
import pynetbox
import yaml
 
ENV_FILE = "yml_examples/env.yml"
 
with open(ENV_FILE) as file:
    env_vars = yaml.safe_load(file)
     
nb_conn = pynetbox.api(url=env_vars['netbox_url'])
 
token = nb_conn.create_token(env_vars['username'], env_vars['password'])

query_device = nb_conn.dcim.devices.get(name="CEOS1")

dev_type = nb_conn.dcim.device_types.get(query_device.device_type.id)

print(dev_type)
print(type(dev_type))

# queried_device_type = nb_conn.dcim.device.type.get(slug="ceos")

temp = [
    {
      "id": 16,
      "url": "http://3.18.147.34:8000/api/dcim/device-types/16/",
      "display": "CEOS",
      "manufacturer": {
        "id": 1,
        "url": "http://3.18.147.34:8000/api/dcim/manufacturers/1/",
        "display": "Arista",
        "name": "Arista",
        "slug": "arista"
      },
      "model": "CEOS1",
      "slug": "ceos"
    }
  ]
r = requests.post('http://3.18.147.34:8000/api/extras/config-contexts/', json={
  "name": "ceos1_conf",
  "weight": 1000,
  "description": "ceos1_conf",
  "is_active": "true",
  "data":ceos_config,
  "device_type":temp
},headers=head)
print(f"Status Code: {r.status_code}, Response: {r.json()}")
# print(type(temp))

CEOS1
<class 'pynetbox.models.dcim.DeviceTypes'>
Status Code: 400, Response: {'name': ['config context with this name already exists.']}
